In [192]:
# !pip install numpy
# !pip install pillow
# !pip install tensorflow
# !pip install keras
# !pip install opencv-python
from numpy import expand_dims
import numpy as np
from tensorflow.keras.utils import load_img,img_to_array,image_dataset_from_directory
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from matplotlib import pyplot as plt
import cv2
import pandas as pd 
import os
from PIL import Image
import tensorflow as tf
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.models import Model, Sequential
from keras.applications.mobilenet import MobileNet
from tensorflow.keras.layers.experimental.preprocessing import Rescaling


In [2]:
rescale = Rescaling(scale=1.0/255)

In [3]:
train_ds = image_dataset_from_directory(
    directory='train',
    labels='inferred',
    label_mode='categorical',
    batch_size=1,
    image_size=(128, 128),
    seed=1024
)
train_ds = train_ds.map(lambda image,label:(rescale(image),label))

Found 17416 files belonging to 7 classes.


In [260]:
val_ds = image_dataset_from_directory(
    directory='val',
    labels='inferred',
    label_mode='categorical',
    batch_size=1,
    image_size=(128, 128),
    seed=1024
)
val_ds =  val_ds.map(lambda image,label:(rescale(image),label))

Found 2898 files belonging to 7 classes.


In [4]:
test_ds = image_dataset_from_directory(
    directory='test',
    labels='inferred',
    label_mode='categorical',
    batch_size=1,
    shuffle=False,
    image_size=(128, 128),
    seed=1024
)
# test_ds = test_ds.map(lambda image,label:(rescale(image),label,test_ds.file_paths))
test_ds = test_ds.map(lambda image,label:(rescale(image),label))

Found 8691 files belonging to 7 classes.


In [ ]:
# model
model_MobileNet =  MobileNet(input_shape=(128, 128, 3), include_top=False, weights="imagenet")



# for layer in model_MobileNet.layers:
#     layer.trainable = False

    # Tạo model với input là ảnh, lấy output của VGG16 và làm input của các layers FC thêm vào
output_model_MobileNet = model_MobileNet.output

# Thêm vào các FC layers 
x = Flatten(name='flatten')(output_model_MobileNet)
x = Dense(20, activation='sigmoid', name='fc1')(x)
x = Dropout(0.5)(x)
x = Dense(7, activation='softmax', name='predictions')(x)
model = Model(model_MobileNet.input, x)
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
              metrics='accuracy',
              loss=tf.keras.losses.CategoricalCrossentropy())
model.summary()

In [199]:
# train


cb_early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    min_delta=1e-4,
    patience=5,
    verbose=1,
    mode='auto',
)
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='accuracy', 
                              factor=0.5, 
                              patience=2, 
                              verbose=1, 
                              mode='max', 
                              min_lr=0.00001)

cb_model_checkpoint = tf.keras.callbacks.ModelCheckpoint(
    'checkpoints/MobileNet',
    monitor= 'val_loss',
    verbose=1,
    save_best_only=True,
    mode='auto'
)
history = model.fit(train_ds,
                    epochs=20,
                    validation_data = val_ds,
                    callbacks=[cb_early_stopping, cb_model_checkpoint,reduce_lr],
                    verbose=1)

In [9]:
model_load.evaluate(test_ds)

8691/8691 [==============================] - 53s 6ms/step - loss: 0.3517 - accuracy: 0.9109


[0.3517066240310669, 0.9109423756599426]

In [ ]:
# test 
label = test_ds
# for i,k in test_ds:
#       print(k)
# print(label.info.features)
image = load_img('test/df/ISIC_0024330.jpg_10.jpg',target_size=(128,128))
image2 =  expand_dims(image, 0)
a = model.predict(image2/255)
print(a)
index = np.argmax(a)
print(index)
# print(label[index])

In [5]:
model_load = tf.keras.models.load_model('checkpoints/MobileNet')

# Show the model architecture
model_load.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 128, 128, 3)]     0         
_________________________________________________________________
conv1 (Conv2D)               (None, 64, 64, 32)        864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 64, 64, 32)        128       
_________________________________________________________________
conv1_relu (ReLU)            (None, 64, 64, 32)        0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 64, 64, 32)        288       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, 64, 64, 32)        128       
_________________________________________________________________
conv_dw_1_relu (ReLU)        (None, 64, 64, 32)        0   

In [7]:
new_model = Model(model_load.input,model_load.get_layer("fc1").output)

In [8]:
new_model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 128, 128, 3)]     0         
_________________________________________________________________
conv1 (Conv2D)               (None, 64, 64, 32)        864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 64, 64, 32)        128       
_________________________________________________________________
conv1_relu (ReLU)            (None, 64, 64, 32)        0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 64, 64, 32)        288       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, 64, 64, 32)        128       
_________________________________________________________________
conv_dw_1_relu (ReLU)        (None, 64, 64, 32)        0   

In [9]:
def build_encoder():
    return tf.keras.models.Model(new_model.input, new_model.output)

model_encoder = build_encoder()
model_encoder.summary()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 128, 128, 3)]     0         
_________________________________________________________________
conv1 (Conv2D)               (None, 64, 64, 32)        864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 64, 64, 32)        128       
_________________________________________________________________
conv1_relu (ReLU)            (None, 64, 64, 32)        0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 64, 64, 32)        288       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, 64, 64, 32)        128       
_________________________________________________________________
conv_dw_1_relu (ReLU)        (None, 64, 64, 32)        0   

In [12]:
model_load_InceptionV3 = tf.keras.models.load_model('checkpoints/InceptionV3')

# Show the model architecture
model_load_InceptionV3.summary()

Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            [(None, 128, 128, 3) 0                                            
__________________________________________________________________________________________________
conv2d_282 (Conv2D)             (None, 63, 63, 32)   864         input_4[0][0]                    
__________________________________________________________________________________________________
batch_normalization_282 (BatchN (None, 63, 63, 32)   96          conv2d_282[0][0]                 
__________________________________________________________________________________________________
activation_282 (Activation)     (None, 63, 63, 32)   0           batch_normalization_282[0][0]    
____________________________________________________________________________________________

In [13]:
model_inceptionV3 = Model(model_load_InceptionV3.input,model_load_InceptionV3.get_layer("fc1").output)

In [16]:
model_load_VGG16 = tf.keras.models.load_model('checkpoints/VGG16')

# Show the model architecture
model_load_VGG16.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 128, 128, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 128, 128, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 128, 128, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 64, 64, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 64, 64, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 64, 64, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 32, 32, 128)       0   

In [17]:
model_VGG16 = Model(model_load_VGG16.input,model_load_VGG16.get_layer("fc1").output)

In [19]:
train_image_VGG16 = []

train_image_inceptionV3 = []
train_image_mobilenet = []
train_label_mobilenet = []


for img,labels in train_ds:
    train_image_inceptionV3.append(model_inceptionV3.predict(img).flatten())
    train_image_VGG16.append(model_VGG16.predict(img).flatten())
    train_image_mobilenet.append(model_encoder.predict(img).flatten())
    train_label_mobilenet.append(np.argmax(labels))


In [70]:
train_mix = []
for i in range(0,len(train_label_mobilenet)):
    train_mix.append(np.concatenate((train_image_inceptionV3[i], train_image_VGG16[i],train_image_mobilenet[i]), axis=None))


In [73]:
print(len(train_mix[0]),len(train_label_mobilenet))

60 17416


In [155]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification


clf = RandomForestClassifier(n_estimators=128)
clf.fit(train_mix,  train_label_mobilenet)

RandomForestClassifier(n_estimators=128)

In [158]:
import joblib
loaded_model = joblib.dump(clf,"randomForest128_cnn_mix.pkl")
# result = loaded_model.score(train_image_embeddings, train_labels)
# print(result)

In [12]:
# test_X = []
# test_Y = []
# for img,labels in test_ds:
#     test_X.append(model_encoder.predict(img).flatten())
#     test_Y.append(np.argmax(labels));


In [79]:
test_image_VGG16 = []

test_image_inceptionV3 = []
test_image_mobilenet = []
test_label_mobilenet = []


for img,labels in test_ds:
    test_image_VGG16.append(model_inceptionV3.predict(img).flatten())
    test_image_inceptionV3.append(model_VGG16.predict(img).flatten())
    test_image_mobilenet.append(model_encoder.predict(img).flatten())
    test_label_mobilenet.append(np.argmax(labels))

In [159]:
import numpy as np
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC

clf_2 =SVC(kernel='linear')
clf_2.fit(train_mix,  train_label_mobilenet)

SVC(kernel='linear')

In [160]:
import joblib
loaded_model = joblib.dump(clf_2,"SVM_cnn_mix.pkl")

In [81]:
test_mix = []
for i in range(0,len(test_label_mobilenet)):
    test_mix.append(np.concatenate((test_image_inceptionV3[i], test_image_VGG16[i],test_image_mobilenet[i]), axis=None))


In [157]:
print(clf.score(test_mix, test_label_mobilenet))  # randomForest 128

0.9017374295247957


In [162]:
print(clf_2.score(test_mix, test_label_mobilenet))  # SVM 

0.7458290185249108


In [163]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification


clf_3 = RandomForestClassifier(n_estimators=1024)
clf_3.fit(train_mix,  train_label_mobilenet)

RandomForestClassifier(n_estimators=1024)

In [154]:
print(clf_3.score(test_mix, test_label_mobilenet))


0.9024277988723968


In [ ]:
# !pip install scikit-learn
# !pip install seaborn
from sklearn.metrics import classification_report, confusion_matrix
import pandas as pd
import seaborn as sns


target_names = ['akiec','bcc','bkl','df','mel','nv','vasc'] 
fig, ax = plt.subplots(figsize=(10,10))
sns.heatmap(cm, annot=True, fmt='', xticklabels=target_names, yticklabels=target_names)
plt.ylabel('Actual')
plt.xlabel('Predicted')
plt.show(block=False)

In [ ]:
fig, ax = plt.subplots(figsize=(10,10))
sns.heatmap(cm1, annot=True, fmt='.4f', xticklabels=target_names, yticklabels=target_names)
plt.ylabel('Actual')
plt.xlabel('Predicted')
plt.show(block=False)

In [ ]:
# ----------------------------------------- Mix MobileNet and InceptionV3 -------------- 

In [164]:
train_mix_2 = []
for i in range(0,len(train_label_mobilenet)):
    train_mix_2.append(np.concatenate((train_image_mobilenet[i],train_image_inceptionV3[i]), axis=None))


In [166]:
test_mix_2 = []
for i in range(0,len(test_label_mobilenet)):
    test_mix_2.append(np.concatenate((test_image_mobilenet[i],test_image_inceptionV3[i]), axis=None))


In [165]:
clf_mix_2_1 = RandomForestClassifier(n_estimators=128)
clf_mix_2_1.fit(train_mix_2,  train_label_mobilenet)

RandomForestClassifier(n_estimators=128)

In [167]:
print(clf_mix_2_1.score(test_mix_2, test_label_mobilenet)) 

0.9046139684731331


In [176]:
clf_mix_2_2 = SVC(kernel='linear')
clf_mix_2_2.fit(train_mix_2,  train_label_mobilenet)

SVC(kernel='linear')

In [177]:
print(clf_mix_2_2.score(test_mix_2, test_label_mobilenet)) 

0.9002416292716603


In [ ]:
# ----------------------------------------- Mix MobileNet and VGG16 -------------- 

In [178]:
train_mix_3 = []
for i in range(0,len(train_label_mobilenet)):
    train_mix_3.append(np.concatenate((train_image_mobilenet[i],train_image_VGG16[i]), axis=None))


In [182]:
test_mix_3 = []
for i in range(0,len(test_label_mobilenet)):
    test_mix_3.append(np.concatenate((test_image_mobilenet[i],test_image_VGG16[i]), axis=None))

In [181]:
clf_mix_3_1 = RandomForestClassifier(n_estimators=128)
clf_mix_3_1.fit(train_mix_3,  train_label_mobilenet)

RandomForestClassifier(n_estimators=128)

In [183]:
print(clf_mix_3_1.score(test_mix_3, test_label_mobilenet)) 

0.9059947071683351


In [189]:
clf_mix_3_2 = SVC(kernel='linear')
clf_mix_3_2.fit(train_mix_3,  train_label_mobilenet)

SVC(kernel='linear')

In [190]:
print(clf_mix_3_2.score(test_mix_3, test_label_mobilenet)) 

0.9024277988723968


In [205]:
print(np.array(train_mix_3).shape)
print(np.array(train_label_mobilenet).shape)

(17416, 40)
(17416,)


In [216]:
train_image_VGG16_MLP = []

train_image_inceptionV3_MLP = []
train_image_mobilenet_MLP = []
train_label_mobilenet_MLP = []

for img,labels in train_ds:
    train_image_inceptionV3_MLP.append(model_inceptionV3.predict(img).flatten())
    train_image_VGG16_MLP.append(model_VGG16.predict(img).flatten())
    train_image_mobilenet_MLP.append(model_encoder.predict(img).flatten())
    train_label_mobilenet_MLP.append(labels)


In [261]:
val_image_VGG16_MLP = []

val_image_inceptionV3_MLP = []
val_image_mobilenet_MLP = []
val_label_mobilenet_MLP = []

for img,labels in val_ds:
    val_image_inceptionV3_MLP.append(model_inceptionV3.predict(img).flatten())
    val_image_VGG16_MLP.append(model_VGG16.predict(img).flatten())
    val_image_mobilenet_MLP.append(model_encoder.predict(img).flatten())
    val_label_mobilenet_MLP.append(labels)


In [263]:
val_label_mobilenet_MLP_re = np.array(val_label_mobilenet_MLP).reshape((-1,7))

In [277]:
test_image_VGG16_MLP = []

test_image_inceptionV3_MLP = []
test_image_mobilenet_MLP = []
test_label_mobilenet_MLP = []

for img,labels in test_ds:
    test_image_inceptionV3_MLP.append(model_inceptionV3.predict(img).flatten())
    test_image_VGG16_MLP.append(model_VGG16.predict(img).flatten())
    test_image_mobilenet_MLP.append(model_encoder.predict(img).flatten())
    test_label_mobilenet_MLP.append(labels)

In [278]:
test_label_mobilenet_MLP_re = np.array(test_label_mobilenet_MLP).reshape((-1,7))

In [219]:
train_mix_3 = []
for i in range(0,len(train_label_mobilenet_MLP)):
    train_mix_3.append(np.concatenate((train_image_mobilenet_MLP[i],train_image_VGG16_MLP[i]),axis=None))

In [269]:
val_mix_3 = []
for i in range(0,len(val_label_mobilenet_MLP)):
    val_mix_3.append(np.concatenate((val_image_mobilenet_MLP[i],val_image_VGG16_MLP[i]),axis=None))

In [346]:
test_mix_3 = []
for i in range(0,len(test_label_mobilenet_MLP_re)):
    test_mix_3.append(np.concatenate((test_image_mobilenet_MLP[i],test_image_VGG16_MLP[i]),axis=None))

In [348]:
print(len(test_mix_3))

8691


In [241]:
train_label_mobilenet_MLP_re = np.array(train_label_mobilenet_MLP).reshape((-1,7))
for i in train_label_mobilenet_MLP_re:
    print(np.array(train_mix_3).shape)
    print(tf.stack(train_label_mobilenet_MLP_re).shape)
    break

(17416, 40)
(17416, 7)


In [410]:
mlp = Sequential()
mlp.add(Dense(224,input_dim=40,activation="sigmoid"))
mlp.add(Dropout(0.2))
mlp.add(Dense(7,activation="softmax"))
mlp.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
              metrics='accuracy',
              loss=tf.keras.losses.CategoricalCrossentropy())
mlp.summary()






Model: "sequential_20"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_101 (Dense)            (None, 224)               9184      
_________________________________________________________________
dropout_66 (Dropout)         (None, 224)               0         
_________________________________________________________________
dense_102 (Dense)            (None, 7)                 1575      
Total params: 10,759
Trainable params: 10,759
Non-trainable params: 0
_________________________________________________________________


In [419]:
# train


cb_early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    min_delta=1e-4,
    patience=5,
    verbose=1,
    mode='auto',
)
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='accuracy', 
                              factor=0.5, 
                              patience=2, 
                              verbose=1, 
                              mode='max', 
                              min_lr=0.00001)

cb_model_checkpoint = tf.keras.callbacks.ModelCheckpoint(
    'checkpoints/MLP_mobileNet_Vgg16',
    monitor= 'val_loss',
    verbose=1,
    save_best_only=True,
    mode='auto'
)
history = mlp.fit(np.array(train_mix_3),tf.stack(train_label_mobilenet_MLP_re),batch_size=64,
                      validation_data=(np.array(val_mix_3),val_label_mobilenet_MLP_re),
                    epochs=50,
                    callbacks=[cb_early_stopping, cb_model_checkpoint,reduce_lr],
                    verbose=1)

Epoch 1/50
273/273 [==============================] - 1s 3ms/step - loss: 0.0127 - accuracy: 0.9992 - val_loss: 0.2549 - val_accuracy: 0.9324

Epoch 00001: val_loss improved from inf to 0.25487, saving model to checkpoints\MLP_mobileNet_Vgg16
INFO:tensorflow:Assets written to: checkpoints\MLP_mobileNet_Vgg16\assets
Epoch 2/50
273/273 [==============================] - 1s 3ms/step - loss: 0.0116 - accuracy: 0.9993 - val_loss: 0.2570 - val_accuracy: 0.9317

Epoch 00002: val_loss did not improve from 0.25487
Epoch 3/50
273/273 [==============================] - 1s 3ms/step - loss: 0.0106 - accuracy: 0.9991 - val_loss: 0.2591 - val_accuracy: 0.9313

Epoch 00003: val_loss did not improve from 0.25487

Epoch 00003: ReduceLROnPlateau reducing learning rate to 2.499999936844688e-05.
Epoch 4/50
273/273 [==============================] - 1s 3ms/step - loss: 0.0099 - accuracy: 0.9991 - val_loss: 0.2604 - val_accuracy: 0.9313

Epoch 00004: val_loss did not improve from 0.25487
Epoch 5/50
273/273 [

In [412]:
mlp.evaluate(np.array(test_mix_3),test_label_mobilenet_MLP_re,batch_size=1)

8691/8691 [==============================] - 15s 2ms/step - loss: 0.2480 - accuracy: 0.9305


[0.24795357882976532, 0.9305028319358826]

In [ ]:
# ------------------------ Mix mobileNet and InceptionV3 MLP ------------------

In [413]:
train_mix_4 = []
for i in range(0,len(train_label_mobilenet_MLP)):
    train_mix_4.append(np.concatenate((train_image_mobilenet_MLP[i],train_image_inceptionV3_MLP[i]),axis=None))

In [414]:
val_mix_4 = []
for i in range(0,len(val_label_mobilenet_MLP)):
    val_mix_4.append(np.concatenate((val_image_mobilenet_MLP[i],val_image_inceptionV3_MLP[i]),axis=None))

In [416]:
test_mix_4 = []
for i in range(0,len(test_label_mobilenet_MLP_re)):
    test_mix_4.append(np.concatenate((test_image_mobilenet_MLP[i],test_image_inceptionV3_MLP[i]),axis=None))

In [417]:
mlp4 = Sequential()
mlp4.add(Dense(224,input_dim=40,activation="sigmoid"))
mlp4.add(Dropout(0.2))
mlp4.add(Dense(7,activation="softmax"))
mlp4.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
              metrics='accuracy',
              loss=tf.keras.losses.CategoricalCrossentropy())
mlp4.summary()


Model: "sequential_21"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_103 (Dense)            (None, 224)               9184      
_________________________________________________________________
dropout_67 (Dropout)         (None, 224)               0         
_________________________________________________________________
dense_104 (Dense)            (None, 7)                 1575      
Total params: 10,759
Trainable params: 10,759
Non-trainable params: 0
_________________________________________________________________


In [421]:
cb_early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    min_delta=1e-4,
    patience=5,
    verbose=1,
    mode='auto',
)
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='accuracy', 
                              factor=0.5, 
                              patience=2, 
                              verbose=1, 
                              mode='max', 
                              min_lr=0.00001)

cb_model_checkpoint = tf.keras.callbacks.ModelCheckpoint(
    'checkpoints/MLP_mobileNet_InceptionV3',
    monitor= 'val_loss',
    verbose=1,
    save_best_only=True,
    mode='auto'
)
history = mlp4.fit(np.array(train_mix_4),tf.stack(train_label_mobilenet_MLP_re),batch_size=64,
                      validation_data=(np.array(val_mix_4),val_label_mobilenet_MLP_re),
                    epochs=50,
                    callbacks=[cb_early_stopping, cb_model_checkpoint,reduce_lr],
                    verbose=1)

Epoch 1/50
273/273 [==============================] - 1s 3ms/step - loss: 0.0141 - accuracy: 0.9996 - val_loss: 0.2712 - val_accuracy: 0.9231

Epoch 00001: val_loss improved from inf to 0.27123, saving model to checkpoints\MLP_mobileNet_InceptionV3
INFO:tensorflow:Assets written to: checkpoints\MLP_mobileNet_InceptionV3\assets
Epoch 2/50
273/273 [==============================] - 1s 3ms/step - loss: 0.0137 - accuracy: 0.9996 - val_loss: 0.2718 - val_accuracy: 0.9234

Epoch 00002: val_loss did not improve from 0.27123
Epoch 3/50
273/273 [==============================] - 1s 3ms/step - loss: 0.0132 - accuracy: 0.9997 - val_loss: 0.2722 - val_accuracy: 0.9234

Epoch 00003: val_loss did not improve from 0.27123

Epoch 00003: ReduceLROnPlateau reducing learning rate to 1e-05.
Epoch 4/50
273/273 [==============================] - 1s 3ms/step - loss: 0.0129 - accuracy: 0.9996 - val_loss: 0.2728 - val_accuracy: 0.9237

Epoch 00004: val_loss did not improve from 0.27123
Epoch 5/50
273/273 [====

In [422]:
mlp4.evaluate(np.array(test_mix_4),test_label_mobilenet_MLP_re,batch_size=1)

8691/8691 [==============================] - 15s 2ms/step - loss: 0.2520 - accuracy: 0.9302


[0.252002477645874, 0.9301576614379883]

In [ ]:
# ------------------------ Mix mobileNet and  InceptionV3 and VGG16 and MLP ------------------

In [424]:
train_mix_5 = []
for i in range(0,len(train_label_mobilenet_MLP)):
    train_mix_5.append(np.concatenate((train_image_mobilenet_MLP[i],train_image_inceptionV3_MLP[i],train_image_VGG16_MLP[i]),axis=None))

In [432]:
val_mix_5 = []
for i in range(0,len(val_label_mobilenet_MLP)):
    val_mix_5.append(np.concatenate((val_image_mobilenet_MLP[i],val_image_inceptionV3_MLP[i],val_image_VGG16_MLP[i]),axis=None))

In [433]:
test_mix_5 = []
for i in range(0,len(test_label_mobilenet_MLP_re)):
    test_mix_5.append(np.concatenate((test_image_mobilenet_MLP[i],test_image_inceptionV3_MLP[i],test_image_VGG16_MLP[i]),axis=None))

In [428]:
print(train_mix_5[2].shape)

(60,)


In [526]:
mlp5 = Sequential()
mlp5.add(Dense(224,input_dim=60,activation="sigmoid"))
mlp5.add(Dropout(0.2))
mlp5.add(Dense(7,activation="softmax"))
mlp5.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
              metrics='accuracy',
              loss=tf.keras.losses.CategoricalCrossentropy())
mlp5.summary()


Model: "sequential_59"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_204 (Dense)            (None, 2401)              146461    
_________________________________________________________________
dropout_128 (Dropout)        (None, 2401)              0         
_________________________________________________________________
dense_205 (Dense)            (None, 7)                 16814     
Total params: 163,275
Trainable params: 163,275
Non-trainable params: 0
_________________________________________________________________


In [525]:
cb_early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    min_delta=1e-4,
    patience=5,
    verbose=1,
    mode='auto',
)
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='accuracy', 
                              factor=0.5, 
                              patience=2, 
                              verbose=1, 
                              mode='max', 
                              min_lr=0.00001)

cb_model_checkpoint = tf.keras.callbacks.ModelCheckpoint(
    'checkpoints/MLP_mobileNet_InceptionV3_VGG16',
    monitor= 'val_loss',
    verbose=1,
    save_best_only=True,
    mode='auto'
)
history = mlp5.fit(np.array(train_mix_5),tf.stack(train_label_mobilenet_MLP_re),batch_size=64,
                      validation_data=(np.array(val_mix_5),val_label_mobilenet_MLP_re),
                    epochs=50,
                    callbacks=[cb_early_stopping, cb_model_checkpoint,reduce_lr],
                    verbose=1)

Epoch 1/50
273/273 [==============================] - 1s 3ms/step - loss: 1.2905 - accuracy: 0.7219 - val_loss: 0.7926 - val_accuracy: 0.9358

Epoch 00001: val_loss improved from inf to 0.79261, saving model to checkpoints\MLP_mobileNet_InceptionV3_VGG16
INFO:tensorflow:Assets written to: checkpoints\MLP_mobileNet_InceptionV3_VGG16\assets
Epoch 2/50
273/273 [==============================] - 1s 2ms/step - loss: 0.4223 - accuracy: 0.9983 - val_loss: 0.3912 - val_accuracy: 0.9369

Epoch 00002: val_loss improved from 0.79261 to 0.39118, saving model to checkpoints\MLP_mobileNet_InceptionV3_VGG16
INFO:tensorflow:Assets written to: checkpoints\MLP_mobileNet_InceptionV3_VGG16\assets
Epoch 3/50
273/273 [==============================] - 1s 2ms/step - loss: 0.1649 - accuracy: 0.9994 - val_loss: 0.2746 - val_accuracy: 0.9375

Epoch 00003: val_loss improved from 0.39118 to 0.27462, saving model to checkpoints\MLP_mobileNet_InceptionV3_VGG16
INFO:tensorflow:Assets written to: checkpoints\MLP_mobi

In [523]:
mlp5.evaluate(np.array(test_mix_5),test_label_mobilenet_MLP_re,batch_size=1)

8691/8691 [==============================] - 14s 2ms/step - loss: 0.1983 - accuracy: 0.9424


[0.19833913445472717, 0.942354142665863]

In [570]:
mlp6 = Sequential()
mlp6.add(Dense(1407,input_dim=60,activation="sigmoid"))
mlp6.add(Dropout(0.2))
mlp6.add(Dense(7,activation="softmax"))
mlp6.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
              metrics='accuracy',
              loss=tf.keras.losses.CategoricalCrossentropy())
mlp6.summary()


Model: "sequential_76"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_241 (Dense)            (None, 1407)              85827     
_________________________________________________________________
dropout_148 (Dropout)        (None, 1407)              0         
_________________________________________________________________
dense_242 (Dense)            (None, 7)                 9856      
Total params: 95,683
Trainable params: 95,683
Non-trainable params: 0
_________________________________________________________________


In [571]:
cb_early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    min_delta=1e-4,
    patience=5,
    verbose=1,
    mode='auto',
)
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='accuracy', 
                              factor=0.5, 
                              patience=2, 
                              verbose=1, 
                              mode='max', 
                              min_lr=0.00001)

cb_model_checkpoint = tf.keras.callbacks.ModelCheckpoint(
    'checkpoints/MLP_mobileNet_InceptionV3_VGG16_2',
    monitor= 'val_loss',
    verbose=1,
    save_best_only=True,
    mode='auto'
)
history = mlp6.fit(np.array(train_mix_5),tf.stack(train_label_mobilenet_MLP_re),batch_size=64,
                      validation_data=(np.array(val_mix_5),val_label_mobilenet_MLP_re),
                    epochs=50,
                    callbacks=[cb_early_stopping, cb_model_checkpoint,reduce_lr],
                    verbose=1)

Epoch 1/50
273/273 [==============================] - 1s 4ms/step - loss: 0.9674 - accuracy: 0.8377 - val_loss: 0.4827 - val_accuracy: 0.9393

Epoch 00001: val_loss improved from inf to 0.48265, saving model to checkpoints\MLP_mobileNet_InceptionV3_VGG16_2
INFO:tensorflow:Assets written to: checkpoints\MLP_mobileNet_InceptionV3_VGG16_2\assets
Epoch 2/50
273/273 [==============================] - 1s 3ms/step - loss: 0.1818 - accuracy: 0.9995 - val_loss: 0.2646 - val_accuracy: 0.9389

Epoch 00002: val_loss improved from 0.48265 to 0.26463, saving model to checkpoints\MLP_mobileNet_InceptionV3_VGG16_2
INFO:tensorflow:Assets written to: checkpoints\MLP_mobileNet_InceptionV3_VGG16_2\assets
Epoch 3/50
273/273 [==============================] - 1s 3ms/step - loss: 0.0640 - accuracy: 0.9997 - val_loss: 0.2220 - val_accuracy: 0.9393

Epoch 00003: val_loss improved from 0.26463 to 0.22200, saving model to checkpoints\MLP_mobileNet_InceptionV3_VGG16_2
INFO:tensorflow:Assets written to: checkpoint

In [572]:
mlp6.evaluate(np.array(test_mix_5),test_label_mobilenet_MLP_re,batch_size=1)

8691/8691 [==============================] - 15s 2ms/step - loss: 0.2012 - accuracy: 0.9427


[0.20117460191249847, 0.9426993727684021]